# 特征缩放

什么是特征缩放？

特征缩放(feature scaling): 改变特征的取值范围，缩放到统一的区间，例如$[0, 1]$.

为什么要进行特征缩放？

数据集包含众多特征，每个特征的尺度(scale)不同，有的特征的单位是小时，有的特征的单位是公里，尺度不同也意味着变化的范围不同，有的特征的波动非常大，有的非常小。对大部分机器学习算法而言，特征取值越大或波动越大，在模型中获得的权重就越大，结果导致预测精度降低，最好的处理办法是将所有特征的尺度缩放到统一的区间。

如何用sklearn实现特征缩放？

sklearn提供了几个通用的接口：

1. MinMaxScaler
2. RobustScaler
3. StandardScaler

参考：

* [sklearn官方文档](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-scaler)
* [如何使用MinMaxScaler和StandardScaler](https://machinelearningmastery.com/standardscaler-and-minmaxscaler-transforms-in-python/)
* [sklearn实现Scale,Standardize,Normalize](https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02)

## 1. MinMaxScaler

MinMaxScaler: 将特征尺度缩放到统一的区间，通常选择$[0, 1]$。

数学逻辑：对于给定的特征，计算其最大值和最小值，然后计算每个观测值在这个区间的所处的位置。

$$y_i = \frac{x_i - min(x)}{max(x) - min(x)}$$

* $y_i$是经过缩放的第i个观测值
* $x_i$是第i个原始观测值
* min(x)和max(x)是最小值和最大值

**MinMaxScaler只会改变特征的取值范围，不会改变分布**

运用MinMaxScaler的最佳实践：

1. 先在训练集调用MinMaxScaler，用训练集的数据计算最大值和最小值
2. 缩放训练集的数据
3. 缩放检验集的数据

In [9]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# 假设训练集包含两个特征，尺度不同
X_train = np.array([
    [100, 0.2],
    [80, 0.25],
    [70, 0.15],
    [150, 0.33],
    [200, 0.54],
    [120, 0.25],
    [135, 0.52],
    [136, 0.42],
    [210, 0.16],
    [90, 0.15]
])

# 创建scaler对象，用户可选择缩放的区间，一般选择[0,1]
scaler = MinMaxScaler(feature_range=(0, 1))

# 拟合训练集
scaler.fit(X_train)

# 缩放训练集
X_train_scaled = scaler.transform(X_train)

# 缩放检验集
X_test = np.array([
    [185, 0.25],
    [150, 0.55]
])
X_test_scaled = scaler.transform(X_test)

# 查看结果
print(X_train)
print(X_train_scaled)
print(X_test_scaled)

[[1.00e+02 2.00e-01]
 [8.00e+01 2.50e-01]
 [7.00e+01 1.50e-01]
 [1.50e+02 3.30e-01]
 [2.00e+02 5.40e-01]
 [1.20e+02 2.50e-01]
 [1.35e+02 5.20e-01]
 [1.36e+02 4.20e-01]
 [2.10e+02 1.60e-01]
 [9.00e+01 1.50e-01]]
[[0.21428571 0.12820513]
 [0.07142857 0.25641026]
 [0.         0.        ]
 [0.57142857 0.46153846]
 [0.92857143 1.        ]
 [0.35714286 0.25641026]
 [0.46428571 0.94871795]
 [0.47142857 0.69230769]
 [1.         0.02564103]
 [0.14285714 0.        ]]
[[0.82142857 0.25641026]
 [0.57142857 1.02564103]]


## 2. RobustScaler

RobustScaler: 利用中位数和四分位距缩放特征，能够在某种程度上抵消异常值的影响。

数学逻辑：对于给定的特征，每个观测值先减去中位数，再除以四分位距。

$$y_i = \frac{x_i - median(x)}{quantile(x, 0.75) - quantile(x, 0.25)}$$

* $y_i$是经过缩放的第i个观测值
* $x_i$是第i个原始观测值
* $median(x)$是中位数
* $quantile(x, k)$计算第k分位数

RobustScaler默认使用四分位距（75%分位数减25%分位数）作为缩放因子，但可以根据实际情况调整，例如计算90%分位数和10%分位数之差。

In [12]:
import numpy as np
from sklearn.preprocessing import RobustScaler

# 假设训练集包含两个特征，尺度不同
X_train = np.array([
    [100, 0.2],
    [80, 0.25],
    [70, 0.15],
    [150, 0.33],
    [200, 0.54],
    [120, 0.25],
    [135, 0.52],
    [136, 0.42],
    [210, 0.16],
    [90, 0.15]
])

# 创建scaler对象，默认使用四分位距作为缩放因子
scaler = RobustScaler(quantile_range=(25.0, 75.0))

# 拟合训练集
scaler.fit(X_train)

# 缩放训练集
X_train_scaled = scaler.transform(X_train)

# 缩放检验集
X_test = np.array([
    [185, 0.25],
    [150, 0.55]
])
X_test_scaled = scaler.transform(X_test)

# 查看结果
print(X_train)
print(X_train_scaled)
print(X_test_scaled)

[[1.00e+02 2.00e-01]
 [8.00e+01 2.50e-01]
 [7.00e+01 1.50e-01]
 [1.50e+02 3.30e-01]
 [2.00e+02 5.40e-01]
 [1.20e+02 2.50e-01]
 [1.35e+02 5.20e-01]
 [1.36e+02 4.20e-01]
 [2.10e+02 1.60e-01]
 [9.00e+01 1.50e-01]]
[[-0.50925926 -0.21978022]
 [-0.87962963  0.        ]
 [-1.06481481 -0.43956044]
 [ 0.41666667  0.35164835]
 [ 1.34259259  1.27472527]
 [-0.13888889  0.        ]
 [ 0.13888889  1.18681319]
 [ 0.15740741  0.74725275]
 [ 1.52777778 -0.3956044 ]
 [-0.69444444 -0.43956044]]
[[1.06481481 0.        ]
 [0.41666667 1.31868132]]


## 3. StandardScaler

StandardScaler: 计算特征的标准分数，令缩放数据的均值为0，方差为1。如果特征近似服从正态分布，那么缩放后的取值范围（大部分数据）将落入$[-3, 3]$区间。

给定一个特征，每个观测值先减去样本均值，再除以标准差：

$$y_i = \frac{x_i - \overline{x}}{s}$$

* $y_i$是经过缩放的第i个观测值
* $x_i$是原始的第i个观测值
* $\overline{x}$是特征的样本均值
* $s$是特征的样本标准差

StandardScaler假设特征服从正态分布，如果不满足这个假设，即便可以对特征进行缩放，也无法获得满意的结果。

In [14]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# 假设训练集包含两个特征，尺度不同
X_train = np.array([
    [100, 0.2],
    [80, 0.25],
    [70, 0.15],
    [150, 0.33],
    [200, 0.54],
    [120, 0.25],
    [135, 0.52],
    [136, 0.42],
    [210, 0.16],
    [90, 0.15]
])

# 创建scaler对象
scaler = StandardScaler()

# 拟合训练集
scaler.fit(X_train)

# 缩放训练集
X_train_scaled = scaler.transform(X_train)

# 缩放检验集
X_test = np.array([
    [185, 0.25],
    [150, 0.55]
])
X_test_scaled = scaler.transform(X_test)

# 查看结果
print(X_train)
print(X_train_scaled)
print(X_test_scaled)

[[1.00e+02 2.00e-01]
 [8.00e+01 2.50e-01]
 [7.00e+01 1.50e-01]
 [1.50e+02 3.30e-01]
 [2.00e+02 5.40e-01]
 [1.20e+02 2.50e-01]
 [1.35e+02 5.20e-01]
 [1.36e+02 4.20e-01]
 [2.10e+02 1.60e-01]
 [9.00e+01 1.50e-01]]
[[-0.64345109 -0.68450885]
 [-1.08568552 -0.33166924]
 [-1.30680273 -1.03734846]
 [ 0.46213498  0.23287414]
 [ 1.56772104  1.71480052]
 [-0.20121666 -0.33166924]
 [ 0.13045916  1.57366467]
 [ 0.15257088  0.86798545]
 [ 1.78883826 -0.96678054]
 [-0.8645683  -1.03734846]]
[[ 1.23604522 -0.33166924]
 [ 0.46213498  1.78536844]]


## 4. 结论

我们学习了3种常用的特征缩放技巧，那么实践中应选择哪种方法？

答案取决于数据集的特点。

以下是一些经验的总结：

1. 如果不明确特征的分布，或根本不在乎分布，一般会先使用MinMaxScaler。
2. 如果特征包含异常值，建议使用RobustScaler，比MinMaxScaler能得到更好的效果。
3. 如果特征近似服从正态分布，最好使用StandardScaler。
4. 如果特征的取值范围很小，变异程度较低，又呈现钟形分布，则没必要进行缩放。